In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit'

We can use Hypergraph features for various predictive tasks:

In [2]:
import convokit
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
corpus = convokit.Corpus(filename=convokit.download("reddit-corpus"))

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus


In [ ]:
hc = convokit.HyperConvo(prefix_len=10, min_thread_len=10, include_root=False)
hc.fit_transform(corpus)

In [ ]:
remake_cache = True
if remake_cache:
    with open("hyperconvo_feats.p", "wb") as f:
        hyperconvo_feats = {}
        for convo in corpus.iter_conversations():
            hyperconvo_feats.update(convo.meta["hyperconvo"])
        pickle.dump(hyperconvo_feats, f)
else:
    with open("hyperconvo_feats.p", "rb") as f:
        hyperconvo_feats = pickle.load(f)

In [ ]:
threads = corpus.utterance_threads(include_root=False)

In [ ]:
missing = 0
present = 0
for tlc_id, thread in threads.items():
    if tlc_id not in thread:
        missing += 1
    else:
        present += 1

In [ ]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10, include_root=False)

In [ ]:
len(threads)

In [ ]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)

In [ ]:
motif_counts = hc.retrieve_motif_counts(corpus)

In [ ]:
threads_motifs = hc.retrieve_motifs(corpus)

In [ ]:
threads_motifs['dnppqdj']

In [ ]:
latent_motif_count = {thread_id: hc._latent_motif_count(motif_dict, trans=False)[0] for thread_id, motif_dict in threads_motifs.items()}

In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import normalize, StandardScaler, Normalizer
from sklearn.linear_model import LogisticRegression

In [ ]:
# first generate positive and negative examples based on task

def generate_pos_neg(task: str, post_to_thread_obj, threads, thread_pfxs):
    pos, neg = [], []
    if task == "comment-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos = [root for root in thread_roots if len(threads[root]) >= 15]
            has_neg = [root for root in thread_roots if len(threads[root]) == 10]
            
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    elif task == "commenter-growth":
        for post_id, thread_roots in post_to_thread_obj.items():
            has_pos, has_neg = [], []
            for root in thread_roots:
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
            if has_pos and has_neg:
                pos.append(random.choice(has_pos))
                neg.append(random.choice(has_neg))
    print("- {} positive, {} negative pts for {} task".format(len(pos), len(neg), task))
    
    return pos, neg


In [ ]:
pos_comment_growth, neg_comment_growth = generate_pos_neg("comment-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

In [ ]:
pos_commenter_growth, neg_commenter_growth = generate_pos_neg("commenter-growth", 
                                                          thread_roots_by_self_post,
                                                          threads,
                                                          thread_pfxs
                                                         )

In [ ]:

# extract features for all these examples

# construct the pair_df, flipping between positive and negative

# StandardScaler + LogisticRegression to finish off


In [ ]:
def generate_paired_features(feats, pos, neg):
    X, y = [], []
    flip = True
    
    for idx in range(len(pos)):
        pos_feats = np.array(list(feats[pos[idx]].values()))
        neg_feats = np.array(list(feats[neg[idx]].values()))
        
        if np.isnan(pos_feats).any() or np.isnan(neg_feats).any(): continue
            
        if flip:
            y.append(1)
            diff = pos_feats - neg_feats
        else:
            y.append(0)
            diff = neg_feats - pos_feats
        X.append(diff)
        flip = not flip

    return np.array(X), np.array(y)

In [ ]:
from copy import deepcopy

In [ ]:
hyperconv_motif = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_motif.items():
    feats.update(motif_counts[thread_id])

In [ ]:
hyperconv_paths = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_paths.items():
    feats.update(path_stats[thread_id])

In [ ]:
hyperconv_latent = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_latent.items():
    feats.update(latent_motif_count[thread_id])

In [ ]:
hyperconv_motifall = deepcopy(hyperconvo_feats)
for thread_id, feats in hyperconv_motifall.items():
    feats.update(motif_counts[thread_id])
    feats.update(path_stats[thread_id])
    feats.update(latent_motif_count[thread_id])

In [ ]:
def print_extreme_coefs(clf, feats):
    feats_ordered = list(feats[next(iter(feats))])
    coefs = clf.named_steps['logreg'].coef_[0].tolist()
    
    assert len(feats_ordered) == len(coefs)
    feats_coefs = sorted(list(zip(feats_ordered, coefs)), key=lambda x: x[1], reverse=True)
    
    print("TOP 5 FEATURES")
    for ft, coef in feats_coefs[:5]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM 5 FEATURES")
    for ft, coef in feats_coefs[-5:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2019)

for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("TASK: {}\n".format(task))
    
    pos, neg = generate_pos_neg(task, thread_roots_by_self_post, threads, thread_pfxs)

    X, y = generate_paired_features(hyperconvo_feats, pos, neg)
    X_motifcnt, y_motifcnt = generate_paired_features(motif_counts, pos, neg)
    X_latent, y_latent = generate_paired_features(latent_motif_count, pos, neg)
    X_path, y_path = generate_paired_features(path_stats, pos, neg)
    X_hcmotif, y_hcmotif = generate_paired_features(hyperconv_motif, pos, neg)
    X_hcpath, y_hcpath = generate_paired_features(hyperconv_paths, pos, neg)
    X_hclatent, y_hclatent = generate_paired_features(hyperconv_latent, pos, neg)
    X_all, y_all = generate_paired_features(hyperconv_motifall, pos, neg)
    for X, y, feats, name in [(X, y, hyperconvo_feats, "hyperconv"),
                       (X_motifcnt, y_motifcnt, motif_counts, "motifcount"),
                       (X_latent, y_latent, latent_motif_count, "latentmotif"),
                       (X_path, y_path, path_stats, "motifpaths"),
                       (X_hcmotif, y_hcmotif, hyperconv_motif, "hyperconv-motif"),
                       (X_hcpath, y_hcpath, hyperconv_paths, "hyperconv-paths"),
                       (X_hclatent, y_hclatent, hyperconv_latent, "hyperconv-latent"),
                       (X_all, y_all, hyperconv_motifall, "hyperconvo-motifall")
                      ]:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])
        clf.fit(X_train, y_train)

        train_acc = clf.score(X_train, y_train)
        test_acc = clf.score(X_test, y_test)
        print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))
        print_extreme_coefs(clf, feats)